In [1]:
# set up OpenAI
import openai

# read api key from file
with open('resources/api-keys.txt', 'r') as f:
    api_key = f.read()

openai.api_key = api_key

In [2]:
import pandas as pd

# Prepocessing libraries
import emoji
import re
from langdetect import detect
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
"""
Language detection
"""
def detect_lang(text):
    try:
        return detect(text)
    except:
        return 'unknown'
    
"""
Auto-correction using GPT-3
"""
def gpt_autocorrect(text, t_max):
    prompt_string = "You will be receiving a text. Your task to correct any spelling mistakes that might be present in the text.  " + text

    r = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt_string,
        temperature=0.2,
        max_tokens=t_max,
        n=1,
        stop=None,
        timeout=10,
    )

    return r['choices'][0]['text']

stop_words = set(stopwords.words('english')) # set of stopwords
punc = set(string.punctuation) # set of all special characters
lemma = WordNetLemmatizer() # lemmatizer
tokenizer = WordPunctTokenizer() # tokenizer

"""
Preprocessing function
"""
def preprocess(text, max_tokens):
    print("Correcting spelling mistakes...")
    text = gpt_autocorrect(text, max_tokens)
    print("Preprocessing...")
    # regex to replace all consecutive occurences of punctuations with a single punctuation
    pattern = r'([' + re.escape(''.join(punc)) + r'])\1+'
    text = re.sub(pattern, r'\1', ''.join(text))

    tokens = tokenizer.tokenize(text.lower())
    # keep token if it is not an emoji
    tokens = [token for token in tokens if emoji.is_emoji(token) == False]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if token not in punc]
    tokens = [lemma.lemmatize(token) for token in tokens]
    print("Story processed!")

    processed_text = ' '.join(tokens)
    return processed_text

Simple test for `gpt_autocorrect()`

In [3]:
correction = gpt_autocorrect("i am perrty bad at speling and grammer, i fel so awfel abt it", 20)
print(correction)

correction = gpt_autocorrect("I am pretty bad at spelling and grammar, I feel so awful about it.", 20)
print(correction)



I am pretty bad at spelling and grammar, I feel so awful about it.


I am pretty bad at spelling and grammar, I feel so awful about it.


In [4]:
# read the data
file = open("data/stories.csv", "r")
stories_array = []

for line in file:
    stories_array.append(line)

file.close()

data = pd.DataFrame(stories_array, columns=['story'])

Demo: `gpt_autocorrect()` and `clean()` on a story

In [5]:
print(data['story'][3494])
print(gpt_autocorrect(data['story'][3494], 500))
print("\n")
print(preprocess(data['story'][3494], 500))

I am an investigative journalist and did a research on the Sugar Mummy scam circus in Singapore. They all operate the same way. No one is what they say they are. I contacted 6 of the agents on Locanto and other sites via WhatsApp and they were all scammers. They might change names but one thing is for 100% sure. You will be scammed! Basically they have a pre-paid phone card with a generic profile photo. They all asure you they are not scammers. After giving them you name, age and civil status they will ask for 300-500 SGDs for a fee. They only accept bank transfer. Then when you have payed this they ask for 1400-1900 SGD for further fees and insurance. They promise you a BMW and a monthly salary of at least 10500 SGD and so on. My conclusion is "DON´T PAY ANYTHING" They are all scammers/fraudsters/liers. Don´t fall for any sweet talk or promises, you will be fooled and no sugar mummy is at the end of the rainbow. No matter who they say they are or that they have lots of clients that re

Solution: add regex to `preprocess()` to catch these cases.

#### Data preprocessing

In [6]:
# add language column
data["language"] = data["story"].apply(detect_lang)

# filter out non-english stories
data = data[data["language"] == "en"]

# drop language column
data = data.drop(columns=["language"])

# get story with highest number of tokens (for GPT-3)
max_tokens = int(data["story"].apply(lambda story: len(word_tokenize(story))).max()) + 50 # 50 for margin of error

In [7]:
# cleaning two stories

two_clean_stories = data['story'][0:2].apply(lambda story: preprocess(story, max_tokens))
print(two_clean_stories)


Correcting spelling mistakes...
Preprocessing...
Story processed!
Correcting spelling mistakes...
Preprocessing...
Story processed!
0    accepted friend request facebook common friend...
1    good morning aaron received application notice...
Name: story, dtype: object


In [8]:
# clean all stories
clean_stories = data['story'].apply(lambda story: preprocess(story, max_tokens))
clean_stories.to_csv("data/clean_stories.csv", index=False)

Correcting spelling mistakes...
Preprocessing...
Story processed!
Correcting spelling mistakes...
Preprocessing...
Story processed!
Correcting spelling mistakes...
Preprocessing...
Story processed!
Correcting spelling mistakes...
